In [1]:
import pandas as pd
import re
from io import StringIO
from ydata_profiling import ProfileReport
import datetime as dt

c:\Users\rudra\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
with open('table.html', 'r') as fileObj:
    table_html = fileObj.read()

table_html = re.sub(r'</?div[^>]*>', '', table_html)
table_html = re.sub(r'</?a[^>]*>', '', table_html)
table_html = re.sub(r'<img[^>]*alt="([^"]*)"[^>]*>', r'\1', table_html)

with open('table.html', 'w') as fileObj:
    fileObj.write(table_html)

with open('table2.html', 'r') as fileObj:
    table_html2 = fileObj.read()

table_html2 = re.sub(r'</?div[^>]*>', '', table_html2)
table_html2 = re.sub(r'</?a[^>]*>', '', table_html2)
table_html2 = re.sub(r'<img[^>]*alt="([^"]*)"[^>]*>', r'\1', table_html2)

with open('table2.html', 'w') as fileObj:
    fileObj.write(table_html2)

df = pd.DataFrame(*pd.read_html(StringIO(table_html))).drop(columns = 'Due').rename(columns={'P' : 'Priority', 'T' : 'Type', 'Key' : 'IssueKey'})
df = pd.concat((
    df,
    pd.DataFrame(*pd.read_html(StringIO(table_html2))).drop(columns = 'Due').rename(columns={'P' : 'Priority', 'T' : 'Type', 'Key' : 'IssueKey'})
)).drop_duplicates().reset_index().drop(columns='index')

In [7]:
df['Created'] = pd.to_datetime(df['Created'])
df['Updated'] = pd.to_datetime(df['Updated'])

daysSince = lambda x : (dt.date.today() - x.date()).days
df['Days Since Created'] = df['Created'].apply(daysSince)
df['Days Since Update'] = df['Updated'].apply(daysSince)

df['Created'] = df['Created'].astype(str)
df['Updated'] = df['Updated'].astype(str)

C:\Users\rudra\AppData\Local\Temp\ipykernel_4832\2766477355.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Created'] = pd.to_datetime(df['Created'])
C:\Users\rudra\AppData\Local\Temp\ipykernel_4832\2766477355.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Updated'] = pd.to_datetime(df['Updated'])


In [8]:
df['Priority Level'] = df['Priority'].apply(lambda x : {'Lowest' : 0, 'Low' : 1, 'Medium' : 2, 'High' : 3, 'Highest' : 4}[x])

In [9]:
df.to_excel('Table.xlsx')

In [8]:
profileReport = ProfileReport(df)
profileReport.to_file('Profile Report.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 66.22it/s]
